In [3]:
# import libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import copy

In [2]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats

In [15]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

In [4]:
# import data and make copy
data = pd.read_csv('Data_rating_112_airlines.csv').copy()

In [5]:
# check first few rows
data.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,aegean-airlines,/airline-reviews/aegean-airlines,Aegean Airlines customer review,P Vlogianitis,Australia,2015-08-01,"Flight to Larnaca was a joy. Generous legroom,...",A321,FamilyLeisure,Business Class,Athens to Larnaca,10.0,5.0,5.0,5.0,NaN,3.0,NaN,5.0,1
1,aegean-airlines,/airline-reviews/aegean-airlines,Aegean Airlines customer review,Eric Botha,United Kingdom,2015-07-28,"Flight on time, row 1 - Friendly staff and ver...",A321,Couple Leisure,Business Class,Athens to London,8.0,4.0,5.0,4.0,2.0,4.0,NaN,4.0,1
2,aegean-airlines,/airline-reviews/aegean-airlines,Aegean Airlines customer review,Nathan Vermeulen,Belgium,2015-07-26,Very good flight with Aegean. The boarding was...,A320-232,FamilyLeisure,Economy,Brussels to Heraklion,9.0,4.0,5.0,4.0,NaN,4.0,NaN,4.0,1
3,aegean-airlines,/airline-reviews/aegean-airlines,Aegean Airlines customer review,N Sunder,United States,2015-07-21,"Brand new A320, interior spotless. Crew immacu...",A320,Solo Leisure,Economy,Mykonos to Athens,8.0,4.0,4.0,NaN,NaN,4.0,NaN,4.0,1
4,aegean-airlines,/airline-reviews/aegean-airlines,Aegean Airlines customer review,Dimitrios Chrysos-Gklavas,Greece,2015-07-21,I booked a premium economy ticket so I can hav...,Dash 8 Q400,Solo Leisure,Economy,Athens to Santorini,8.0,4.0,5.0,4.0,NaN,2.0,NaN,3.0,1


In [6]:
# check summary statistics
data.describe()

,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
count,32275.000000,31415.000000,31417.000000,31020.000000,28980.000000,2003.000000,534.000000,35314.000000,35609.000000
mean,6.033369,3.109056,3.335742,2.827176,2.463285,2.719421,2.284644,3.181090,0.544188
std,3.244976,1.392464,1.529093,1.568518,1.697691,1.566430,1.552541,1.517891,0.498051
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,3.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000
50%,7.000000,3.000000,4.000000,3.000000,3.000000,3.000000,1.000000,4.000000,1.000000
75%,9.000000,4.000000,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000
max,10.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000


In [7]:
# check additional information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35609 entries, 0 to 35608
Data columns (total 20 columns):
airline_name                     35609 non-null object
link                             35609 non-null object
title                            35609 non-null object
author                           35609 non-null object
author_country                   35396 non-null object
date                             35609 non-null object
content                          35609 non-null object
aircraft                         1153 non-null object
type_traveller                   2164 non-null object
cabin_flown                      34138 non-null object
route                            2128 non-null object
overall_rating                   32275 non-null float64
seat_comfort_rating              31415 non-null float64
cabin_staff_rating               31417 non-null float64
food_beverages_rating            31020 non-null float64
inflight_entertainment_rating    28980 non-null float64
ground_se

### 1. GROUP BY TRAVELLER TYPE

In [12]:
# group data by traveller type and count number of rows for recommended reviews
data.groupby('type_traveller')['recommended'].count()

type_traveller
Business          329
Couple Leisure    544
FamilyLeisure     559
Solo Leisure      732
Name: recommended, dtype: int64

In [13]:
# group data by traveller type and calculate average for recommended reviews
data.groupby('type_traveller')['recommended'].mean()

type_traveller
Business          0.431611
Couple Leisure    0.443015
FamilyLeisure     0.423971
Solo Leisure      0.524590
Name: recommended, dtype: float64

null hypothesis: There is no difference in means of recommended rating between passengers based on traveller type.

alternative hypothesis: There is difference between means of recommended rating between passengers based on traveller type.

In [5]:
# ANOVA using scipy.stats
stats.f_oneway(data['recommended'][data['type_traveller'] == 'Business'], 
               data['recommended'][data['type_traveller'] == 'Couple Leisure'],
               data['recommended'][data['type_traveller'] == 'FamilyLeisure'],
               data['recommended'][data['type_traveller'] == 'Solo Leisure'])

F_onewayResult(statistic=5.619120334066824, pvalue=0.0007784730472595241)

The F-statistic = 5.619 and the p-value = 0.0008 which is indicating that there is an overall significant effect of traveller type on recommended rating. 

In [8]:
# ANOVA using statsmodels (regression formula)
results_traveller = ols('recommended ~ C(type_traveller)', data=data).fit()
results_traveller.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            recommended   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     5.619
Date:                Thu, 14 Nov 2019   Prob (F-statistic):           0.000778
Time:                        09:58:06   Log-Likelihood:                -1556.6
No. Observations:                2164   AIC:                             3121.
Df Residuals:                    2160   BIC:                             3144.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.4316      0.027     15.745      0.000       0.378       0.485
C(type_traveller)[T.Couple Leisure]     0.0114      0.035      0.328      0.743      -0.057       0.080
C(type_traveller)[T.FamilyLeisure]     -0.0076      0.035     -0.221      0.825      -0.075       0.060
C(type_traveller)[T.Solo Leisure]       0.0930      0.033      2.817      0.005       0.028       0.158
==============================================================================
Omnibus:                     7974.880   Durbin-Watson:                   1.521
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              349.580
Skew:                           0.144   Prob(JB):                     1.23e-76
Kurtosis:                       1.052   Cond. No.                         6.04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Overall the model is significant, F(3,2160)= 5.619, p = 0.0008. This tells us that there is a significant difference in the group means. The coefficients are the difference in means between the control group and the respective group listed. The Intercept is the mean for the Business traveller type group. Family Leisure group has a negative coefficient, which means it has an average value lower than Business group. In other words, on average Family Leisure will less likely to recommend airline than Business type of traveller.
Looking at the p-values (P>|t|), we can see the difference between the Business group and Solo Leisure group is significant, p = 0.005, but the difference between the Business group and any other two groups is not, (p = 0.743 and p = 0.825).


In [9]:
# getting ANOVA table
aov_table_traveller = sm.stats.anova_lm(results_traveller, typ=2)
aov_table_traveller

,sum_sq,df,F,PR(>F)
C(type_traveller),4.167677,3.0,5.61912,0.000778
Residual,534.020863,2160.0,NaN,NaN


The type traveller row is the between groups effect which is the overall experimental effect. The sum of squares for the model (sum_sq = 4.168) is how much variance is explained by the model. The model explains a significant amount of variance, F(3,2160)= 5.619, p < 0.05. The Residual row is the unexplained variance in the data (sum_sq = 534.021). In this case, the unexplained variance represents the individual differences in recommendation and different reactions to a traveller type.


#### 1.1. Calculating Model Effect Size

The effect size tells us how much of an impact the experiment will have in the real world.


In [10]:
# extract sum_sq column values from ANOVA table
aov_table_traveller.sum_sq[0], aov_table_traveller.sum_sq[1]

(4.16767706703947, 534.0208626741805)

In [71]:
# calculating eta squared - R2
# formula: R2 and eta squared = SS_m/SS_r
R2_and_eta_squared = aov_table_traveller.sum_sq[0]/(aov_table_traveller.sum_sq[0]+aov_table_traveller.sum_sq[1])
print('The current model accounts for '+ str(round(R2_and_eta_squared,4)*100) + '% of the variance in contributing to recommended rating.')

The current model accounts for 0.77% of the variance in contributing to recommended rating.


In [12]:
# calculate the mean squares
# formulas: 
# MS_m = SS_m/df_m, MS_r = SS_r/df_r
MSm_traveller = aov_table_traveller.sum_sq[0]/aov_table_traveller.df[0]
MSr_traveller = aov_table_traveller.sum_sq[1]/aov_table_traveller.df[1]
print(MSm_traveller, 'is the average amount of variance explained by the current model.') 
print(MSr_traveller, 'is the average amount of variance unexplained by the current model.')

1.3892256890131567 is the average amount of variance explained by the current model.
0.24723188086767617 is the average amount of variance unexplained by the current model.


In [13]:
# the function to calculate the effect sizes, the mean squares and then updates the table
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(aov_table_traveller)

,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(type_traveller),4.167677,3.0,1.389226,5.61912,0.000778,0.007744,0.006363
Residual,534.020863,2160.0,0.247232,NaN,NaN,NaN,NaN


From the anova_table we get the same values for the average amounts (mean_sq) as calculated above (MSm_traveller and MSr_traveller). F-statistic represents the ratio of MSm_traveller to MSr_traveller. eta_sq is also called R2 (calculated above), and can be interpreted as the amount of explained variance, as well as an effect size measure.

#### 1.2. Homogeneity of Variance

We can use the Levene’s test to test for equal variances between groups.

In [14]:
stats.levene(data['recommended'][data['type_traveller'] == 'Business'], 
               data['recommended'][data['type_traveller'] == 'Couple Leisure'],
               data['recommended'][data['type_traveller'] == 'FamilyLeisure'],
               data['recommended'][data['type_traveller'] == 'Solo Leisure'])

LeveneResult(statistic=1.3126570838357872, pvalue=0.26853625706562273)

Levene’s test for homogeneity of variance is not statistically significant, which indicates that the groups have equal variances.

To test between groups, we need to do some post-hoc testing where we can compare all groups against each other. 

#### 1.3. Tukey’s HSD Post-hoc comparison

Using Tukey’s HSD to test which groups differ.

In [34]:
# create a list with unique type travellers
list_traveller = list(data['type_traveller'].unique())

In [38]:
list_traveller

['FamilyLeisure', 'Couple Leisure', 'Solo Leisure', 'Business', nan]

In [39]:
# create dataframe where traveller type doesn't have an empty row
data_traveller = data.loc[data['type_traveller'].isin(['FamilyLeisure', 'Couple Leisure', 'Solo Leisure', 'Business'])]

In [40]:
# Tukey HSD post-hoc comparison test between different group means
mc_traveller = MultiComparison(data_traveller['recommended'], data_traveller['type_traveller'])
mc_results_traveller = mc_traveller.tukeyhsd()
print(mc_results_traveller)

        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1         group2     meandiff p-adj   lower  upper  reject
-------------------------------------------------------------------
      Business Couple Leisure   0.0114    0.9 -0.0779 0.1007  False
      Business  FamilyLeisure  -0.0076    0.9 -0.0965 0.0812  False
      Business   Solo Leisure    0.093 0.0252  0.0081 0.1778   True
Couple Leisure  FamilyLeisure   -0.019    0.9  -0.096 0.0579  False
Couple Leisure   Solo Leisure   0.0816 0.0198  0.0092 0.1539   True
 FamilyLeisure   Solo Leisure   0.1006 0.0018  0.0288 0.1724   True
-------------------------------------------------------------------


The group1 and group2 columns are the groups being compared, the meandiff column is the difference in means of the two groups being calculated as group2 – group1, the lower/upper columns are the lower/upper boundaries of the 95% confidence interval, and the reject column states whether or not the null hypothesis should be rejected.
Based on that we can reject the null hypothesis for three comparisons: There is a difference in means for recommended rating between Business and Couple Leisure traveller type. There is also a difference in means for recommended rating between Business and Family Leisure traveller type. We can also reject for comparison between Couple Leisure and Family Leisure traveller type.


### 2. GROUP BY FLIGHT CLASS

In [14]:
# group data by flight class and count number of rows for recommended reviews
data.groupby('cabin_flown')['recommended'].count()

cabin_flown
Business Class      5962
Economy            25862
First Class          841
Premium Economy     1473
Name: recommended, dtype: int64

In [18]:
# group data by flight class and count number of rows for recommended reviews
data.groupby('cabin_flown')['recommended'].sum()

cabin_flown
Business Class      4108
Economy            13409
First Class          537
Premium Economy      792
Name: recommended, dtype: int64

In [15]:
# group data by flight class and calculate average for recommended reviews
data.groupby('cabin_flown')['recommended'].mean()

cabin_flown
Business Class     0.689031
Economy            0.518483
First Class        0.638526
Premium Economy    0.537678
Name: recommended, dtype: float64

null hypothesis: There is no difference in means of recommended rating between passengers based on the cabin flown (flight class).

alternative hypothesis: There is difference between means of recommended rating passengers based on the cabin flown (flight class).

In [6]:
# ANOVA using scipy.stats
stats.f_oneway(data['recommended'][data['cabin_flown'] == 'Business Class'], 
               data['recommended'][data['cabin_flown'] == 'Economy'],
               data['recommended'][data['cabin_flown'] == 'First Class'],
               data['recommended'][data['cabin_flown'] == 'Premium Economy'])

F_onewayResult(statistic=202.47664595260855, pvalue=3.5377794911393746e-130)

The F-statistic = 202.477 and the p-value = 3.5377794911393746e-130 which is indicating that there is an overall significant effect of the cabin flown on recommended rating. 

In [44]:
# ANOVA using statsmodels (regression formula)
results_class = ols('recommended ~ C(cabin_flown)', data=data).fit()
results_class.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            recommended   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     202.5
Date:                Thu, 14 Nov 2019   Prob (F-statistic):          3.54e-130
Time:                        11:22:43   Log-Likelihood:                -24290.
No. Observations:               34138   AIC:                         4.859e+04
Df Residuals:                   34134   BIC:                         4.862e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.6890      0.006    107.928      0.000       0.677       0.702
C(cabin_flown)[T.Economy]            -0.1705      0.007    -24.082      0.000      -0.184      -0.157
C(cabin_flown)[T.First Class]        -0.0505      0.018     -2.781      0.005      -0.086      -0.015
C(cabin_flown)[T.Premium Economy]    -0.1514      0.014    -10.552      0.000      -0.179      -0.123
==============================================================================
Omnibus:                   124709.880   Durbin-Watson:                   1.818
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5319.327
Skew:                          -0.196   Prob(JB):                         0.00
Kurtosis:                       1.107   Cond. No.                         9.14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Overall the model is significant, F(3,34134)= 202.5, p = 3.54e-130. This tells us that there is a significant difference in the group means. The Intercept is the mean for the Business flight class group. Since all coefficients are negative, all groups have lower mean compared to the Business class group. Looking at the p-values (P>|t|), we can see the differences between the Business class group and any other three groups are significant.

#### 2.1. Calculating Model Effect Size

In [45]:
# getting ANOVA table
aov_table_class = sm.stats.anova_lm(results_class, typ=2)
aov_table_class

,sum_sq,df,F,PR(>F)
C(cabin_flown),147.602565,3.0,202.476646,3.537779e-130
Residual,8294.398489,34134.0,NaN,NaN


The cabin flown row is the between groups effect which is the overall experimental effect. The sum of squares for the model (sum_sq = 147.602) is how much variance is explained by the model. The model explains a significant amount of variance, F(3,34134)= 202.477, p < 0.05. The Residual row is the unexplained variance in the data (sum_sq = 8294.398). In this case, the unexplained variance represents the individual differences in recommended rating and different reactions to a cabin flown.

In [46]:
# updateting ANOVA table
anova_table(aov_table_class)

,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(cabin_flown),147.602565,3.0,49.200855,202.476646,3.537779e-130,0.017484,0.017397
Residual,8294.398489,34134.0,0.242995,NaN,NaN,NaN,NaN


In addition to the above explained values, eta_sq means the model accounts for 1.75% of the variance in contributing to recommended rating. The mean_sq = 49.201 is the average amount of variance explained by the model, the mean_sq = 0.243 is the average amount of variance unexplained by the model.

#### 2.2. Homogeneity of Variance

In [47]:
stats.levene(data['recommended'][data['cabin_flown'] == 'Business Class'], 
               data['recommended'][data['cabin_flown'] == 'Economy'],
               data['recommended'][data['cabin_flown'] == 'First Class'],
               data['recommended'][data['cabin_flown'] == 'Premium Economy'])

LeveneResult(statistic=202.47664595260858, pvalue=3.5377794911393746e-130)

Levene’s test for homogeneity of variance is statistically significant, which indicates that the groups don't have equal variances.

#### 2.3. Tukey’s HSD Post-hoc comparison

In [48]:
# check unique values for cabin flown
data['cabin_flown'].unique()

array(['Business Class', 'Economy', 'Premium Economy', 'First Class', nan],
      dtype=object)

In [49]:
# create a dataframe without NaN values
data_class = data.loc[data['cabin_flown'].isin(['Business Class', 'Economy', 'Premium Economy', 'First Class'])]

In [50]:
# test which groups differ
# Tukey HSD post-hoc comparison test between different group means
mc_class = MultiComparison(data_class['recommended'], data_class['cabin_flown'])
mc_results_class = mc_class.tukeyhsd()
print(mc_results_class)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1          group2     meandiff p-adj   lower   upper  reject
---------------------------------------------------------------------
Business Class         Economy  -0.1705  0.001 -0.1887 -0.1524   True
Business Class     First Class  -0.0505 0.0278 -0.0972 -0.0039   True
Business Class Premium Economy  -0.1514  0.001 -0.1882 -0.1145   True
       Economy     First Class     0.12  0.001  0.0757  0.1644   True
       Economy Premium Economy   0.0192 0.4669 -0.0147  0.0531  False
   First Class Premium Economy  -0.1008  0.001 -0.1556 -0.0461   True
---------------------------------------------------------------------


From the comparison test, we can reject the null hypothesis for comparison between Economy and Premium Economy class. Which means there is not a statistically significant difference in the mean values for recommended rating between these two groups/classes.

### 3. GROUP BY TRAVELLER TYPE AND FLIGHT CLASS (2-way ANOVA)

In [9]:
# group data by traveller type and flight class and count number of rows for recommended reviews
data.groupby(['type_traveller', 'cabin_flown'])['recommended'].count()

type_traveller  cabin_flown    
Business        Business Class     129
                Economy            176
                First Class         19
                Premium Economy      4
Couple Leisure  Business Class      79
                Economy            424
                First Class         11
                Premium Economy     24
FamilyLeisure   Business Class      35
                Economy            507
                First Class         13
                Premium Economy      4
Solo Leisure    Business Class      99
                Economy            578
                First Class         32
                Premium Economy     19
Name: recommended, dtype: int64

In [11]:
# group data by traveller type and flight class and count number of rows for recommended reviews
data.groupby(['type_traveller', 'cabin_flown'])['recommended'].mean()

type_traveller  cabin_flown    
Business        Business Class     0.635659
                Economy            0.284091
                First Class        0.473684
                Premium Economy    0.250000
Couple Leisure  Business Class     0.632911
                Economy            0.389151
                First Class        0.818182
                Premium Economy    0.666667
FamilyLeisure   Business Class     0.742857
                Economy            0.402367
                First Class        0.384615
                Premium Economy    0.500000
Solo Leisure    Business Class     0.787879
                Economy            0.461938
                First Class        0.718750
                Premium Economy    0.789474
Name: recommended, dtype: float64

Since we have already tested for significance for each factor separately, we can also test for the significance of the interaction between traveller type and cabin flown.
If the interaction is significant, the sole main effects of traveller type or cabin flown are not really interpretable by themselves since the significant interaction indicates that the effect of factor traveller type depends on the level of factor traveller type and the level of factor cabin flown, and vise versa. If the interaction effect between factors traveller type and factor cabin flown is not significant, then we remove the interaction from the model and use test for significant main effects by themselves.

null hypothesis: The factors, traveller type and cabin flown, are independent. There is no interaction between them.

alternative hypothesis: The factors, traveller type and cabin flown, are dependent. There is interaction between them.


In [53]:
# ANOVA using statsmodels (regression formula)
results_travell_class_1 = ols('recommended ~ C(type_traveller) * C(cabin_flown)', data=data).fit()
results_travell_class_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            recommended   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     10.23
Date:                Thu, 14 Nov 2019   Prob (F-statistic):           4.52e-24
Time:                        15:26:57   Log-Likelihood:                -1482.8
No. Observations:                2153   AIC:                             2998.
Df Residuals:                    2137   BIC:                             3088.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 0.6357      0.043     14.929      0.000       0.552       0.719
C(type_traveller)[T.Couple Leisure]                                      -0.0027      0.069     -0.040      0.968      -0.138       0.133
C(type_traveller)[T.FamilyLeisure]                                        0.1072      0.092      1.163      0.245      -0.074       0.288
C(type_traveller)[T.Solo Leisure]                                         0.1522      0.065      2.356      0.019       0.026       0.279
C(cabin_flown)[T.Economy]                                                -0.3516      0.056     -6.272      0.000      -0.461      -0.242
C(cabin_flown)[T.First Class]                                            -0.1620      0.119     -1.363      0.173      -0.395       0.071
C(cabin_flown)[T.Premium Economy]                                        -0.3857      0.246     -1.571      0.116      -0.867       0.096
C(type_traveller)[T.Couple Leisure]:C(cabin_flown)[T.Economy]             0.1078      0.082      1.322      0.186      -0.052       0.268
C(type_traveller)[T.FamilyLeisure]:C(cabin_flown)[T.Economy]              0.0111      0.101      0.109      0.913      -0.188       0.210
C(type_traveller)[T.Solo Leisure]:C(cabin_flown)[T.Economy]               0.0256      0.077      0.333      0.739      -0.125       0.176
C(type_traveller)[T.Couple Leisure]:C(cabin_flown)[T.First Class]         0.3472      0.196      1.773      0.076      -0.037       0.731
C(type_traveller)[T.FamilyLeisure]:C(cabin_flown)[T.First Class]         -0.1963      0.197     -0.996      0.319      -0.583       0.190
C(type_traveller)[T.Solo Leisure]:C(cabin_flown)[T.First Class]           0.0928      0.154      0.602      0.547      -0.210       0.395
C(type_traveller)[T.Couple Leisure]:C(cabin_flown)[T.Premium Economy]     0.4194      0.270      1.552      0.121      -0.110       0.949
C(type_traveller)[T.FamilyLeisure]:C(cabin_flown)[T.Premium Economy]      0.1428      0.354      0.403      0.687      -0.552       0.837
C(type_traveller)[T.Solo Leisure]:C(cabin_flown)[T.Premium Economy]       0.3873      0.274      1.414      0.157      -0.150       0.924
==============================================================================
Omnibus:                     9581.415   Durbin-Watson:                   1.584
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              275.265
Skew:                           0.169   Prob(JB):                     1.69e-60
Kurtosis:                       1.281   Cond. No.                         72.7
==================================================================

Overall the model is significant, F(15,2137)= 10.23, p = 4.52e-24. This tells us that there is a significant difference in the group means. The Intercept is the mean for the Business flight class and Business traveller type. The only group that has lower average recommended rating in comparison with Business travellers in Business flight class, is a group of Family travellers who fly in First flight class. Looking at the p-values (P>|t|) for combination groups, we can see the differences between the control group and any other group are not significant.

In [55]:
# Creates the ANOVA table
anova_travell_class_1 = sm.stats.anova_lm(results_travell_class_1, typ= 2)
anova_travell_class_1

,sum_sq,df,F,PR(>F)
C(type_traveller),6.840535,3.0,9.749635,2.178153e-06
C(cabin_flown),29.343894,3.0,41.823083,2.930372e-26
C(type_traveller):C(cabin_flown),2.353190,9.0,1.117980,3.460803e-01
Residual,499.787013,2137.0,NaN,NaN


The interaction term is not significant (p > 0.05). This indicates that there is no interaction effect between the type of traveller and the cabin flown (flight class) on the mean value for recommended rating.
Since this is not significant, the interaction term is to be removed from the model and it needs to be re-ran so we can look at the main effects of each factor independently.


In [56]:
# ANOVA using statsmodels (regression formula)
results_travell_class_3 = ols('recommended ~ C(type_traveller) + C(cabin_flown)', data=data).fit()
results_travell_class_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            recommended   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     23.88
Date:                Thu, 14 Nov 2019   Prob (F-statistic):           1.75e-27
Time:                        15:41:40   Log-Likelihood:                -1487.9
No. Observations:                2153   AIC:                             2990.
Df Residuals:                    2146   BIC:                             3029.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.6083      0.032     18.846      0.000       0.545       0.672
C(type_traveller)[T.Couple Leisure]     0.0904      0.035      2.596      0.010       0.022       0.159
C(type_traveller)[T.FamilyLeisure]      0.1038      0.035      2.950      0.003       0.035       0.173
C(type_traveller)[T.Solo Leisure]       0.1736      0.033      5.242      0.000       0.109       0.238
C(cabin_flown)[T.Economy]              -0.3147      0.030    -10.487      0.000      -0.374      -0.256
C(cabin_flown)[T.First Class]          -0.1003      0.062     -1.621      0.105      -0.222       0.021
C(cabin_flown)[T.Premium Economy]      -0.0570      0.073     -0.778      0.437      -0.201       0.087
==============================================================================
Omnibus:                     9438.353   Durbin-Watson:                   1.586
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              278.854
Skew:                           0.162   Prob(JB):                     2.80e-61
Kurtosis:                       1.267   Cond. No.                         10.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Overall, the model is statistically significant.  This tells us that there is a significant difference in the group means. The Intercept is the mean for the Business flight class and Business traveller type. All cabin flown groups have lower average values compared to the Intercept. Looking at the p-values (P>|t|) only First class and Premium Economy groups aren't statistically significant (less likely to affect the recommended rating).

In [57]:
# create the ANOVA table
anova_travell_class_3 = sm.stats.anova_lm(results_travell_class_3, typ= 2)
anova_travell_class_3

,sum_sq,df,F,PR(>F)
C(type_traveller),6.840535,3.0,9.744814,2.192404e-06
C(cabin_flown),29.343894,3.0,41.802399,2.995672e-26
Residual,502.140204,2146.0,NaN,NaN


Each factor has an independent significant effect on the recommended rating mean. 

In [58]:
# calculate the effect size
anova_table(anova_travell_class_3)

,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(type_traveller),6.840535,3.0,2.280178,9.744814,2.192404e-06,0.012707,0.011398
C(cabin_flown),29.343894,3.0,9.781298,41.802399,2.995672e-26,0.054510,0.053183
Residual,502.140204,2146.0,0.233989,NaN,NaN,NaN,NaN


Each factor, traveller type and cabin flown, has an effect on the recommended rating mean.

To find if there are any differences between type of traveller or cabin flown we can do post-hoc testing, which we already this above. In the above post-hoc testing we discover there are some differences between the traveller type groups as well as between cabin flown groups (Economy and Premium Economy).


### 4. ONLY BUSINESS FLIGHT CLASS

In [59]:
# getting data just for the business traveller type
data_business = data.loc[data['type_traveller'] == 'Business']

null hypothesis: There is no difference in means of recommended rating between flight classes for Business traveller type passengers.

alternative hypothesis: There is a difference between the means of recommended rating between flight classes for Business traveller type passengers.


In [61]:
# using scipy.stats, the method needed is stats.f_oneway()
stats.f_oneway(data_business['recommended'][data_business['cabin_flown'] == 'Business Class'], 
               data_business['recommended'][data_business['cabin_flown'] == 'Economy'],
               data_business['recommended'][data_business['cabin_flown'] == 'First Class'],
               data_business['recommended'][data_business['cabin_flown'] == 'Premium Economy'])

F_onewayResult(statistic=14.21537537152095, pvalue=1.0024981166881016e-08)

In [60]:
# ANOVA using statsmodels (regression formula)
results_business = ols('recommended ~ C(cabin_flown)', data=data_business).fit()
results_business.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            recommended   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     14.22
Date:                Thu, 14 Nov 2019   Prob (F-statistic):           1.00e-08
Time:                        20:09:54   Log-Likelihood:                -214.80
No. Observations:                 328   AIC:                             437.6
Df Residuals:                     324   BIC:                             452.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.6357      0.041     15.406      0.000       0.554       0.717
C(cabin_flown)[T.Economy]            -0.3516      0.054     -6.472      0.000      -0.458      -0.245
C(cabin_flown)[T.First Class]        -0.1620      0.115     -1.407      0.161      -0.389       0.065
C(cabin_flown)[T.Premium Economy]    -0.3857      0.238     -1.621      0.106      -0.854       0.082
==============================================================================
Omnibus:                     2356.915   Durbin-Watson:                   1.735
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.212
Skew:                           0.241   Prob(JB):                     2.75e-07
Kurtosis:                       1.593   Cond. No.                         10.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Overall the model is statistically significant, F(3, 324)=14.22, p<0.05. This tells us that there is a significant difference in the group means. The Intercept is the mean for the Business flight class. All cabin flown groups have lower average values compared to the Intercept. Looking at the p-values (P>|t|) First class and Premium Economy groups aren't statistically significant (less likely to affect the recommended rating).

In [62]:
# create anova table
aov_table_business = sm.stats.anova_lm(results_business, typ=2)
aov_table_business

,sum_sq,df,F,PR(>F)
C(cabin_flown),9.366125,3.0,14.215375,1.002498e-08
Residual,71.158266,324.0,NaN,NaN


The cabin_flown row is the between groups effect which is the overall experimental effect. The sum of squares for the model (sum_s q= 9.366) is how much variance is explained by the model. The model explains a significant amount of variance, F(3,324)= 14.215, p < 0.05. The residual row is the unexplained variance in the data (sum_sq = 71.158).

#### 4.1 Calculating Model Effect Size

In [63]:
# updating anova table
anova_table(aov_table_business)

,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(cabin_flown),9.366125,3.0,3.122042,14.215375,1.002498e-08,0.116314,0.107838
Residual,71.158266,324.0,0.219624,NaN,NaN,NaN,NaN


In addition to the above explained values, eta_sq means the model accounts for 11.63% of the variance in contributing to recommended rating. The mean_sq = 3.122 is the average amount of variance explained by the model, the mean_sq = 0.22 is the average amount of variance unexplained by the model.

#### 4.2 Homogeneity of Variance

In [64]:
# using the Levene’s test to test for equal variances between groups
stats.levene(data_business['recommended'][data_business['cabin_flown'] == 'Business Class'], 
               data_business['recommended'][data_business['cabin_flown'] == 'Economy'],
               data_business['recommended'][data_business['cabin_flown'] == 'First Class'],
               data_business['recommended'][data_business['cabin_flown'] == 'Premium Economy'])

LeveneResult(statistic=1.4209821827334477, pvalue=0.23652329183204437)

Levene’s test for homogeneity of variance is not significant which indicates that the groups have equal variances.

#### 4.3. Tukey’s HSD Post-hoc comparison

In [67]:
# create a dataframe without NaN values
data_business_class = data_business.loc[data_business['cabin_flown'].isin(['Business Class', 'Economy', 'Premium Economy', 'First Class'])]

In [68]:
# comparing groups means
mc_business = MultiComparison(data_business_class['recommended'], data_business_class['cabin_flown'])
mc_business_results = mc_business.tukeyhsd()
print(mc_business_results)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1          group2     meandiff p-adj   lower   upper  reject
---------------------------------------------------------------------
Business Class         Economy  -0.3516  0.001 -0.4918 -0.2113   True
Business Class     First Class   -0.162 0.4962 -0.4594  0.1354  False
Business Class Premium Economy  -0.3857  0.369 -1.0001  0.2288  False
       Economy     First Class   0.1896 0.3387 -0.1027  0.4819  False
       Economy Premium Economy  -0.0341    0.9 -0.6461  0.5779  False
   First Class Premium Economy  -0.2237 0.7984 -0.8895  0.4421  False
---------------------------------------------------------------------


The reject column tells us that we should reject the null hypothesis for all comparisons except for the comparison between Business class and Economy class. There is a statistically significant difference between the mean recommended rating between Business class and Economy class. Business class rates/recommends a significantly higher recommended rating than Economy class.